In [6]:
import wandb
import numpy as np

# Log in to WandB
wandb.login()

# Replace with your entity, project, and run ID
entity = "nlp_ls"
projects = ["dspro2-predicting-country", "dspro2-predicting-region", "dspro2-predicting-coordinates"]
run_ids = "*"  # or []

# Value to add to the summary
summary_key = None  # "test_data_run_id"
summary_value = None  # "w1098m89"

# Push best Validation Accuracy * and Validation Distance and Validation Loss as Logs
push_best = True

# Access the run
api = wandb.Api()
for project in projects:
    project_run_ids = run_ids
    if project_run_ids == "*":
        # Get all runs of the project
        project_run_ids = []
        print(f"Getting run id's for project {project}")
        for run in api.runs(f"{entity}/{project}"):
            project_run_ids.append(run.id)
        print(f"Found {len(project_run_ids)} runs")
    for run_id in project_run_ids:
        print(f"Checking run {run_id}")
        if summary_key is not None and summary_value is not None:
            run = api.run(f"{entity}/{project}/{run_id}")

            # Update the summary
            run.summary[summary_key] = summary_value
            run.summary.update()

            print(f"Successfully updated summary of run {run_id} with {summary_key}: {summary_value}")

        # Push best Validation Accuracy * and Validation Distance and Validation Loss to summary
        # Technically, this is different from pushing it after training, because there it will use the values of the best epoch, however, this is a good approximation
        if push_best:
            run = api.run(f"{entity}/{project}/{run_id}")
            # Get all metrics in summary that start with "Validation"
            metrics = run.summary.keys()
            validation_metrics = [k for k in metrics if k.lower().startswith("validation")]
            best_validation_metrics = {}
            # For every validation metric, push the best value from the history
            for metric in validation_metrics:
                best_key = f"Best {metric}"
                if best_key in metrics:
                    continue
                all_values = run.history()[metric]  # numpy array
                all_values_without_nan = all_values[~np.isnan(all_values)]
                if len(all_values_without_nan) > 0:
                    best_value = np.max(all_values_without_nan) if "accuracy" in metric.lower() or "correct" in metric.lower() else np.min(all_values_without_nan)
                else:
                    best_value = np.nan
                best_validation_metrics[best_key] = best_value
                # Update the summary
                run.summary[best_key] = best_value
            if len(best_validation_metrics) > 0:
                run.summary.update()
                print(f"Successfully updated summary of run {run_id} with {best_validation_metrics}")
            else:
                print(f"Skipped updating summary of run {run_id}")

Getting run id's for project dspro2-predicting-region
Found 55 runs
Checking run 7f0wprn5
Skipped updating summary of run 7f0wprn5
Checking run hl40ix17
Skipped updating summary of run hl40ix17
Checking run xr2tkule
Skipped updating summary of run xr2tkule
Checking run udglvnth
Skipped updating summary of run udglvnth
Checking run 6518mmnp
Skipped updating summary of run 6518mmnp
Checking run ov5mhb9g
Skipped updating summary of run ov5mhb9g
Checking run cf8ecfwe
Skipped updating summary of run cf8ecfwe
Checking run j3zik6vp
Successfully updated summary of run j3zik6vp with {'Best Validation Accuracy Top 3 Country': 0.055, 'Best Validation Distance (km)': 4829.95891947686, 'Best Validation Accuracy Top 3': 0.018544303797468353, 'Best Validation Accuracy Top 1 Country': 0.02677215189873418, 'Best Validation Accuracy Top 1': 0.011265822784810127, 'Best Validation Accuracy Top 5': 0.023607594936708862, 'Best Validation Loss': 21.82173878633523, 'Best Validation Accuracy Top 5 Country': 0.